## tools in langchain


### tool wiki


In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)

In [3]:
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki.name

'wikipedia'

### custom tool

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size =1000,chunk_overlap=200).split_documents(docs)
db = FAISS.from_documents(documents,OllamaEmbeddings(model="llama3.2"))
retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000011522FB36B0>, search_kwargs={})

In [9]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever,"langsmith_search","Search for information about Langsmith. For any questions about Langsmith, you must use this tool.")

In [10]:
retriever_tool.name

'langsmith_search'

### tool arxiv

In [11]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

### list of all tools

In [29]:
from langchain.agents import load_tools
tools = [wiki,arxiv]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Administrator\\Desktop\\langchain\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200))]

### loading openai api keys and openai model

In [15]:
from dotenv import load_dotenv

load_dotenv()


# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo=0125",temperature=0)


True

### using ollama model

In [16]:
from langchain_ollama.llms import  OllamaLLM

llm = OllamaLLM(model="llama3.2")

### prompt

In [17]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### agent

In [30]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm=llm,tools=tools,prompt=prompt)

### agent executor


In [35]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [32]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [33]:
agent_executor.invoke({"input":'Tell me about Langsmith'})



> Entering new AgentExecutor chain...


TypeError: Client.generate() got an unexpected keyword argument 'tools'

In [37]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [39]:
from langchain_ollama import OllamaLLM
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain import hub

# Step 1: Set up Ollama LLM
llm = OllamaLLM(model="llama3")  # Replace with your desired model name

# Step 2: Set up tools (e.g., Wikipedia)
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)

# Step 3: Define tools
tools = [wiki_tool]


prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages
# Step 4: Create agent
agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Step 5: Set up agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Step 6: Invoke agent
response = agent_executor.invoke({"input": "Tell me about LangSmith"})
print(response)




> Entering new AgentExecutor chain...


TypeError: Client.generate() got an unexpected keyword argument 'tools'